In [75]:
import torch
from torch import nn
from matplotlib import pyplot as plt
%matplotlib inline

In [76]:
words = open("webhacking.txt", "r").read().splitlines()
# with open('webhacking.txt', 'r', encoding='utf-8') as f:
#     words = f.read()

In [77]:
chars = sorted(list(set(''.join(words))))
stoi = {s: i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos) + 1
print(itos)
print(vocab_size)


{1: ' ', 2: '!', 3: '$', 4: '&', 5: '(', 6: ')', 7: ',', 8: '-', 0: '.', 10: '0', 11: '1', 12: '2', 13: '3', 14: '5', 15: '6', 16: '7', 17: '9', 18: ':', 19: ';', 20: '?', 21: 'A', 22: 'B', 23: 'C', 24: 'D', 25: 'E', 26: 'F', 27: 'G', 28: 'H', 29: 'I', 30: 'J', 31: 'K', 32: 'L', 33: 'M', 34: 'N', 35: 'O', 36: 'P', 37: 'Q', 38: 'R', 39: 'S', 40: 'T', 41: 'U', 42: 'V', 43: 'W', 44: 'Y', 45: 'a', 46: 'b', 47: 'c', 48: 'd', 49: 'e', 50: 'f', 51: 'g', 52: 'h', 53: 'i', 54: 'j', 55: 'k', 56: 'l', 57: 'm', 58: 'n', 59: 'o', 60: 'p', 61: 'q', 62: 'r', 63: 's', 64: 't', 65: 'u', 66: 'v', 67: 'w', 68: 'x', 69: 'y', 70: 'z', 71: 'â', 72: '€', 73: '™'}
74


In [78]:
# build the dataset
# context length: how many characters do we take to predict the next one?
import random
block_size = 8


def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix]  # crop and append


  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

random.seed(42)
# random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte = build_dataset(words[n2:])     # 10%


torch.Size([11829, 8]) torch.Size([11829])
torch.Size([1479, 8]) torch.Size([1479])
torch.Size([1452, 8]) torch.Size([1452])


In [ ]:
frick = torch.randn((73,25))

frick[72]

In [ ]:

g = torch.Generator().manual_seed(2147483647)

C = torch.randn((27, n_embed), generator=g)
w1 = torch.randn(n_embed * block_size, n_hidden, generator=g) * (5/3)/(n_embed * block_size) **0.5
# b1 = torch.randn([n_hidden]) * 0.01
w2 = torch.randn(n_hidden, vocab_size, generator=g) * 0.01
b2 = torch.randn([vocab_size], generator = g) * 0

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bn_meanrunning = torch.zeros([1, n_hidden])
bn_stdrunning = torch.ones([1, n_hidden])
parameters = [C, w1, w2, b2, bngain,bnbias]
print(sum(p.nelement() for p in parameters))
for i in parameters:
    i.requires_grad = True

In [ ]:
max_steps = 200000
bacth_size = 32
lossi = []

for i in range(max_steps):
    ix = torch.randint(0, Xtr.shape[0], (bacth_size,))

    # ix = ix.
    Xb, Yb = Xtr[ix], Ytr[ix]

    emb = C[Xb]
    # hidden = torch.tanh(emb.view( -1, block_size * n_embed) @ w1 + b1)
    hpreact = emb.view(-1, block_size * n_embed) @ w1
    bnmeani = hpreact.mean(0, keepdim=True)
    bnstdi = hpreact.std(0, keepdim=True)
    hpreact = bngain * ((hpreact - bnmeani) / bnstdi) + bnbias

    hpreact = torch.tanh(hpreact)
    finreact = hpreact @ w2 + b2

    loss = nn.functional.cross_entropy(finreact, Yb)

    
        
    for p in parameters:
        p.grad = None

        
    loss.backward()

    lr = 0.1 if i < 100000 else 0.01

    for p in parameters:
        p.data += - lr * p.grad
    with torch.no_grad():
        bn_meanrunning = 0.999 * bn_meanrunning + 0.001 * bnmeani
        bn_stdrunning = 0.999 * bn_stdrunning + 0.001 * bnstdi
    if i % 10000 == 0:
        print(f"{i:7d}/ {max_steps} : {loss.item():.4f}")
    lossi.append(loss.log10().item())

In [ ]:
with torch.no_grad():

    res = C[Xtr]
    hidden = res.view(-1, n_embed * block_size) @ w1 + b1
    hidden = torch.tanh(hidden)

    bnmean = hidden.mean(0, keepdim=True)
    bnstd = hidden.std(0, keepdim=True)

In [ ]:
bnmean

In [ ]:
# bn_meanrunning

x = torch.tensor([[1,2,3,4],[5,6,7,8],[9,10,11,12]])
y = torch.tensor(([[5,10,15,20]]))

print((x @ y) )

In [ ]:
plt.hist(hidden.view(-1).tolist(), bins = 50);

In [ ]:
plt.figure(figsize=(20,20))

plt.imshow(hpreact.abs() > 0.99 , cmap='gray', interpolation='nearest')

In [ ]:
@torch.no_grad()  # this decorator disables gradient tracking
def split_loss(split):
  x, y = {
      'train': (Xtr, Ytr),
      'val': (Xdev, Ydev),
      'test': (Xte, Yte),
  }[split]
  emb = C[x]  # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1)  # concat into (N, block_size * n_embd)
  hidden = embcat @ w1 + b1
  hidden = bngain * ((hidden - bn_meanrunning) / bn_stdrunning) + bnbias
  hidden = torch.tanh(hidden)
  logits = hidden @ w2 + b2
  loss = nn.functional.cross_entropy(logits, y)
  print(split, loss.item())


split_loss('train')
split_loss('val')


In [79]:
class linear:

    def __init__(self,fan_ins,fan_outs,bias=True):
        self.weight = torch.randn([fan_ins,fan_outs]) / (fan_ins)**0.5
        self.bias = torch.zeros([fan_outs]) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    
class batchnorm1d:

    def __init__(self, dim, eps=1e-5,momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True

        self.gamma = torch.ones([dim])
        self.beta = torch.zeros([dim])

        self.running_mean = torch.zeros([dim])
        self.running_var = torch.ones([dim])

    def __call__(self, x):

        if self.training:
            dim = 0
            if x.ndim == 2:
                dim = (0,1)
            xmean = x.mean(dim, keepdim=True)
            xvar = x.var(dim, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var

        xhat = (x - xmean ) / xvar ** 0.5
        self.out = self.gamma * xhat + self.beta


        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar

        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out


    def parameters(self):
        return []
    
class Embedding:
    def __init__(self, vocab_size, n_embed):
        self.weight = torch.randn([vocab_size, n_embed])

    def __call__(self, x):
        self.out = self.weight[x]
        return self.out
    
    def parameters(self):
        return [self.weight]

class flatten:

    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        b,t,c = x.shape
        rip = x.view(b,t//self.n,c*self.n)
        
        if rip.shape[1] == 1:
            rip = rip.squeeze(1)

        self.out = rip
        return self.out
    
    def parameters(self):
        return []

class sequential:
    def __init__(self, layers):

        self.layers = layers

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [80]:
n_embed = 24
n_hidden = 128

# C = torch.randn([vocab_size, n_embed])
model = sequential([
    Embedding(vocab_size, n_embed),
    flatten(2), linear(n_embed * 2, n_hidden, bias=False), batchnorm1d(n_hidden), Tanh(),
    flatten(2), linear(n_hidden * 2,n_hidden, bias=False), batchnorm1d(n_hidden), Tanh(),
    flatten(2), linear(n_hidden * 2, n_hidden, bias=False), batchnorm1d(n_hidden), Tanh(),
    linear(n_hidden, vocab_size)
])

with torch.no_grad():
    model.layers[-1].weight *= 0.1

    # for layer in layers:
    #     if isinstance(layer, linear):
    #         layer.weight *= 5/3

parameters = model.parameters()
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

83770


In [ ]:
for layer in model.layers:
    print(f"{layer.__class__.__name__} : {tuple(layer.out.shape)}")

In [81]:
max_steps = 1
batch_size = 32
losses = []
ud = []
for i in range(max_steps):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    Xb , Yb = Xtr[ix], Ytr[ix]
    
    # print(Xb[0][1])
    #forward pass


    x = model(Xb)
    print(x.shape,  Yb.shape)
    loss = nn.functional.cross_entropy(x, Yb)
     
    for p in parameters:
        p.grad = None
    loss.backward()

    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    if i % 10000 == 0:
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f}")
    losses.append(loss.log10().item())
    with torch.no_grad():
        ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

    loss = nn.functional.cross_entropy(x, Yb)
    # break
print(loss)


torch.Size([32, 74]) torch.Size([32])
      0/      1: 4.3029
tensor(4.3029, grad_fn=<NllLossBackward0>)


In [ ]:
# visualize histograms
plt.figure(figsize=(20, 4))  # width and height of the plot
legends = []
for i, layer in enumerate(model.layers[:-1]):  # note: exclude the output layer
  if isinstance(layer, Tanh):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, saturated: %.2f%%' %
          (i, layer.__class__.__name__, t.mean(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1].detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__}')
plt.legend(legends)
plt.title('activation distribution')


In [ ]:
plt.figure(figsize=(20, 4))
legends = []
for i, p in enumerate(parameters):
  if p.ndim == 2:
    plt.plot([ud[j][i] for j in range(len(ud))])
    legends.append('param %d' % i)
# these ratios should be ~1e-3, indicate on plot
plt.plot([0, len(ud)], [-3, -3], 'k')
plt.legend(legends);


In [ ]:
for layer in model.layers:
    layer.training = False

In [ ]:

for i in range(200):

    # context  = [random.randint(0, vocab_size-1) for i in range(block_size)]
    context = [0]*block_size
    # print(context)
    out = [i for i in context]

    while True:
        ix = torch.tensor([context])

        # ix = torch.randint(0, Xtr.shape[0], (batch_size,))
        # Xb, Yb = C[Xtr[ix]], Ytr[ix]

        # forward pass

        # x = ix.view(-1, n_embed * block_size)
        x = model(ix)
        probs = nn.functional.softmax(x, dim=1)

        charindex = torch.multinomial(probs, 1)
        context = context[1:] + [charindex]

        out.append(charindex.item())

        if charindex.item() == 0:
            break
    print("".join(itos[i] for i in out))